<a href="https://colab.research.google.com/github/shavinkalu23/7318_assignment_2/blob/main/7318_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchvision.transforms.v2 import ToTensor,Resize,Compose,ColorJitter,RandomRotation,RandomAffine, AugMix,RandomCrop,GaussianBlur,RandomEqualize,RandomHorizontalFlip,RandomVerticalFlip, Normalize
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch.utils.data import random_split

SEED = 42

torch.manual_seed(SEED)

# If you are using CUDA
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# transform = Compose([
#     Resize([32,32]),
#     ToTensor(),
# ])
# Load the entire GTSRB dataset
full_dataset = torchvision.datasets.GTSRB(root='./data',
                                          split='train',
                                          download=True)

# 80-20 split
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Further split training data into training and validation datasets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# def compute_mean_std(loader):
#     mean = 0.
#     std = 0.
#     nb_samples = 0.

#     for data, _ in loader:
#         batch_samples = data.size(0)
#         data = data.view(batch_samples, data.size(1), -1)
#         mean += data.mean(2).sum(0)
#         std += data.std(2).sum(0)
#         nb_samples += batch_samples

#     mean /= nb_samples
#     std /= nb_samples

#     return mean, std

# train_loader_for_stats = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)
# mean, std = compute_mean_std(train_loader_for_stats)
# print(mean,std)

mean = (0.3337, 0.3064, 0.3171)
std =  ( 0.2672, 0.2564, 0.2629)

train_transforms = Compose([
    Resize([32,32]),
    ToTensor(),
    Normalize((mean), (std))
])




validation_transforms = Compose([
    Resize([32,32]),
    ToTensor(),
    Normalize((mean), (std))
])

test_transforms = Compose([
    Resize([32,32]),
    ToTensor(),
    Normalize((mean), (std))
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)


# Apply transformations
#train_dataset = CustomDataset(train_dataset, transform=train_transforms)
val_dataset = CustomDataset(val_dataset, transform=validation_transforms)
test_dataset = CustomDataset(test_dataset, transform=test_transforms)

BS = 64

# Data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BS, shuffle=True, num_workers=2, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=2, pin_memory=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")





100%|██████████| 187490228/187490228 [00:07<00:00, 25310979.16it/s]


Extracting data/gtsrb/GTSRB-Training_fixed.zip to data/gtsrb


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [18]:
class BaseCNN(nn.Module):
    def __init__(self):
        super(BaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 43)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x






model = BaseCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        print('h')
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Finished Training")

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the base model on test images: {100 * correct / total}%")


h
h
h
h


KeyboardInterrupt: ignored

In [ ]:
torch.save(model, '/content/drive/My Drive/base_model.pth')

In [ ]:
# Load the VGG16 model
vgg16 = models.vgg16(pretrained=False)  # We use a non-pretrained model for this task
vgg16.classifier[6] = nn.Linear(4096, 43)  # Change the last layer to have 43 outputs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = vgg16.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Finished Training")


Epoch 1, Loss: 3.5479794589163873
Epoch 2, Loss: 3.415389202767425
Epoch 3, Loss: 3.07171157852923
Epoch 4, Loss: 1.9437988481933264
Epoch 5, Loss: 1.2386082027741758
Epoch 6, Loss: 0.6781395765588724
Epoch 7, Loss: 0.3685031889779248
Epoch 8, Loss: 0.20899475604402934
Epoch 9, Loss: 0.13316944544594422
Epoch 10, Loss: 0.08970553634522987
Finished Training
Accuracy of the VGG16 model on test images: 85.3840063341251%


In [ ]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Models/vgg16.pth')

In [ ]:
 # Load the resnet34 model
resnet34 = models.resnet34(pretrained=False)  # We use a non-pretrained model for this task
resnet34.fc = nn.Linear(resnet34.fc.in_features, 43)  # Change the final fully connected layer to have 43 outputs

model = resnet34.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training
num_epochs = 3
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Finished Training")

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the resnet34 model on test images: {100 * correct / total}%")

torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Models/resnet34.pth')




Epoch 1, Loss: 3.025985900732539
Epoch 2, Loss: 1.6239267143128302
Epoch 3, Loss: 0.6506958918319904
Finished Training
Accuracy of the resnet34 model on test images: 71.1243072050673%


Data augmentation can introduce variance into the dataset and reduce overfitting. Given the GTSRB dataset contains traffic signs which can be captured under various conditions, we can use a few common augmentations:

In [6]:
def training_metrics(positives, data_size, loss):
    acc = positives / data_size
    return loss, acc

def validation_metrics(validation_data, loss_function, model):
    data_size = len(validation_data.dataset)
    correct_predictions = 0
    total_samples = 0
    val_loss = 0

    model.eval()
    with torch.no_grad():
        for step, (input, label) in enumerate(validation_data):
            input, label = input.to(device), label.to(device)
            prediction = model(input)
            loss = loss_function(prediction, label)
            val_loss += loss.item()
            _, predicted = torch.max(prediction, 1)
            correct_predictions += (predicted == label).sum().item()
            total_samples += label.size(0)

    val_acc = correct_predictions / total_samples
    return val_loss / len(validation_data), val_acc


In [7]:
def compile(train_data, validation_data, epochs, loss_function, optimizer, learning_rate_scheduler, model):
    val_acc_list = []
    val_loss_list = []

    train_acc_list = []
    train_loss_list = []

    learning_rate_list = []

    print('Training started ...')
    STEPS = len(train_data)
    for epoch in range(epochs):
        lr = optimizer.param_groups[0]["lr"]
        learning_rate_list.append(lr)
        correct_predictions = 0
        total_examples = 0
        loss_val = 0

        pbar = tqdm(enumerate(train_data), total=STEPS, desc=f"Epoch [{epoch+1}/{epochs}]")
        for step, (input, label) in pbar:
            input, label = input.to(device), label.to(device)
            prediction = model(input)

            _, predicted = torch.max(prediction, 1)
            correct_predictions += (predicted == label).sum().item()
            total_examples += label.size(0)

            loss = loss_function(prediction, label)
            loss_val += loss.item()
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            pbar.set_postfix({
                "Learning Rate": lr,
                "Loss": f"{loss.item():.4f}",
                "Accuracy": f"{correct_predictions/total_examples:.4f}"
            })

        training_loss, training_acc = training_metrics(correct_predictions, total_examples, loss_val / len(train_data))
        train_acc_list.append(training_acc)
        train_loss_list.append(training_loss)

        val_loss, val_acc = validation_metrics(validation_data, loss_function, model)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)

        print(f'Validation Accuracy: {val_acc:.4f}, Validation Loss: {val_loss:.4f}')

        learning_rate_scheduler.step()

    metrics_dict = {
        'train_acc': train_acc_list,
        'train_loss': train_loss_list,
        'val_acc': val_acc_list,
        'val_loss': val_loss_list,
        'learning_rate': learning_rate_list
    }

    return metrics_dict

In [58]:
mean = (0.3337, 0.3064, 0.3171)
std =  ( 0.2672, 0.2564, 0.2629)

train_transforms = Compose([
    RandomHorizontalFlip(),
    RandomRotation(10),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    RandomAffine(degrees=0, translate=(0.1, 0.1)),
    Resize([32,32]),
    ToTensor(),
    Normalize((mean), (std))
])
train_dataset = CustomDataset(train_dataset, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=2, pin_memory=True)

In [4]:
EPOCHS = 10
LEARNING_RATE = 0.0001
INPUT_DIM = 3*32*32
OUTPUT_DIM = 43

# Load the VGG16 model
vgg16 = models.vgg16(pretrained=False)  # We use a non-pretrained model for this task
vgg16.classifier[6] = nn.Linear(4096, 43)  # Change the last layer to have 43 outputs
weight_decay = 0.0005
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = vgg16.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(),lr=LEARNING_RATE)
lr_s = optim.lr_scheduler.LinearLR(optimizer,start_factor=1.0,end_factor=0.5,total_iters=10)
loss = nn.CrossEntropyLoss()



NameError: ignored

In [65]:
met = compile(model =model, train_data=train_loader,validation_data=val_loader,epochs=EPOCHS,loss_function=loss,optimizer=optimizer,learning_rate_scheduler=lr_s)

Training started ...


Epoch [1/10]: 100%|██████████| 267/267 [00:27<00:00,  9.74it/s, Learning Rate=0.0001, Loss=2.3226, Accuracy=0.1150]


Validation Accuracy: 0.2723, Validation Loss: 2.1853


Epoch [2/10]: 100%|██████████| 267/267 [00:27<00:00,  9.78it/s, Learning Rate=9.5e-5, Loss=1.5831, Accuracy=0.3560]


Validation Accuracy: 0.4438, Validation Loss: 1.5734


...

Validation Accuracy: 0.8865, Validation Loss: 0.3124


Epoch [9/10]: 100%|██████████| 267/267 [00:27<00:00,  9.64it/s, Learning Rate=6e-5, Loss=0.3376, Accuracy=0.8897]


Validation Accuracy: 0.9123, Validation Loss: 0.2417


Epoch [10/10]: 100%|██████████| 267/267 [00:27<00:00,  9.68it/s, Learning Rate=5.5e-5, Loss=0.5050, Accuracy=0.9048]


Validation Accuracy: 0.9292, Validation Loss: 0.2029


In [1]:
def plot_metrics(metrics_dict):
    epochs = range(1, len(metrics_dict['train_acc']) + 1)

    # Plot training and validation accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, metrics_dict['train_acc'], label='Training Accuracy', marker='o')
    plt.plot(epochs, metrics_dict['val_acc'], label='Validation Accuracy', marker='o')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, metrics_dict['train_loss'], label='Training Loss', marker='o')
    plt.plot(epochs, metrics_dict['val_loss'], label='Validation Loss', marker='o')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [2]:
plot_metrics(met)

NameError: ignored